In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tifffile
import pickle

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torchvision.models as models

import random
SEED_VAL = 42
random.seed(SEED_VAL)
np.random.seed(SEED_VAL)
torch.manual_seed(SEED_VAL)
torch.cuda.manual_seed_all(SEED_VAL)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
import sys  
sys.path.insert(0, './utils')

from submission import *

In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(device)

cuda


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Dataset

Zindi provides a sample submission as a CSV file, which I now import in order to create my own submission file.

In [6]:
ss = pd.read_csv('SampleSubmission.csv')
ss.head()

,fn,maize streak virus,disease,okukkoola,muwogo,mpeke,mucungwa,greens,garden,mango,bulimi,obuwuka,ebikoola,obulimi,ebisoolisooli,kaamulali,eddagala,beans,omuyembe,leaf,kisaanyi,leaves,butterfly,okuzifuuyira,micungwa,ppaapaali,emboga,kikolo,harvest,olusuku,coffee,super grow,rice,ensujju,okulima,worm,ebbugga,onion,ensigo,plantation,ensukusa,maize stalk borer,amakoola,muceere,insect,pumpkin,spray,namuginga,tomatoes,akatungulu,ground nuts,orange,nursery bed,npk,mangoes,ebitooke,maize,endwadde,lumonde,ensiringanyi,pawpaw,ebijjanjalo,sukumawiki,caterpillars,matooke seedlings,okusaasaana,omuceere,ebikajjo,mulimi,sikungula,watermelon,ennima,akammwanyi,amakungula,ebyobulimi,okugimusa,sow,Pump,ekigimusa,ekirime,okusiga,nfukirira,faamu,okulimibwa,pampu,vegetables,lusuku,ennyaanya,ekisaanyi,bibala,ekikajjo,ennimiro,okukungula,akatunda,eppapaali,endagala,ekijanjaalo,abalimi,plant,Spinach,ndwadde,caterpillar,miceere,farmer,irrigate,nnakati,omusiri,weeding,okusaasaanya,amalagala,ekikoola,ebisooli,munyeera,ekirwadde,enkota,banana,fruit picking,ebibala,dig,doodo,omulimisa,eggobe,seed,crop,okunnoga,obutunda,fruit,endokwa,ebikolo,omucungwa,okufukirira,obulwadde,ebijanjaalo,matooke,pepper,agriculture,afukirira,nnyaanya,medicine,nambaale,cassava,sugarcane,ekimera,kikajjo,obusaanyi,ekibala,ebigimusa,kukungula,nnimiro,ebikongoliro,irish potatoes,ensuku,emicungwa,ekikolo,ejjobyo,omulimi,okusimba,sweet potatoes,okufuuyira,farming instructor,nnasale beedi,passion fruit,ekitooke,ebisaanyi,ekyeya,enva endiirwa,emisiri,emiyembe,amatooke,ebiwuka,farm,ebinyebwa,amappapaali,ebimera,kassooli,harvesting,emmwanyi,akamonde,obumonde,cabbages,akasaanyi,spread,ebirime,drought,kasaanyi,suckers,insects,fertilizer,nakavundira,ekiwojjolo,akawuka,ddagala,ebiwojjolo,obutungulu
0,audio_files/00118N3.wav,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,audio_files/00P0NMV.wav,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,audio_files/01QEEZI.wav,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,audio_files/037YAED.wav,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,audio_files/0382N0Y.wav,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Since I need to use the label mapping which I created in `keyword_spotter_preprocessing.ipynb`, I now import the saved pickle file.

In [7]:
with open('./pickles/labels_numeric_mapping.pkl', 'rb') as file:
    labels_numeric_mapping = pickle.load(file)
    
with open('./pickles/labels_mapping.pkl', 'rb') as file:
    labels_mapping = pickle.load(file)

The mapping now allows me to map the classes to numbers.

In [8]:
ss_num = ss.rename(columns=labels_mapping)
column_order = list(ss_num.columns)
ss_num.head()

,fn,26,188,56,83,9,47,166,36,181,167,160,110,118,23,60,66,72,125,137,14,38,19,77,63,175,111,130,126,127,132,157,173,7,58,123,174,81,109,6,139,184,68,104,39,165,94,24,170,40,156,89,46,129,119,172,131,48,30,51,49,133,141,54,43,10,62,106,147,29,186,61,121,124,91,25,155,183,70,27,161,80,190,8,4,122,179,178,16,71,86,67,31,151,182,59,55,97,65,76,17,140,28,93,84,117,18,35,98,128,189,116,20,78,50,1,159,90,115,99,103,21,41,57,176,5,171,145,101,107,169,2,34,136,42,168,138,3,45,153,32,192,158,15,102,185,11,150,73,154,134,92,164,12,105,152,146,120,53,148,96,75,163,64,187,100,162,144,143,69,13,74,108,135,52,44,87,113,95,191,85,112,33,37,114,142,180,149,79,88,0,82,22,177
0,audio_files/00118N3.wav,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,audio_files/00P0NMV.wav,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,audio_files/01QEEZI.wav,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,audio_files/037YAED.wav,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,audio_files/0382N0Y.wav,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Next, I build the Dataset and the DataLoader.

In [21]:
test_paths = list(ss.fn)
test_paths_tif = [path.split('/')[1].split('.wav')[0] + '.tif' for path in test_paths]

In [22]:
test_ds = Spectrogram_Dataset_Submission(test_paths_tif)

In [23]:
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=10, shuffle=False, num_workers=8)

## Models

Since for each model during training, I saved the weights at the moment of highest accuracy, I can now use those weights for testing the model on the test set.

In [14]:
densenet_testing = models.densenet161(pretrained=True)
num_ftrs = densenet_testing.classifier.in_features
densenet_testing.classifier = nn.Linear(num_ftrs, 193)

densenet_testing.load_state_dict(torch.load('./models/best_model_zindi_densenet_26-11.pt'))

Downloading: "https://download.pytorch.org/models/densenet161-8d451a50.pth" to /root/.cache/torch/hub/checkpoints/densenet161-8d451a50.pth


<All keys matched successfully>

In [15]:
resnet_testing = models.resnet50(pretrained=True)
num_ftrs = resnet_testing.fc.in_features
resnet_testing.fc = nn.Linear(num_ftrs, 193)

resnet_testing.load_state_dict(torch.load('./models/best_model_zindi_resnet_26-11.pt'))

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


<All keys matched successfully>

In [16]:
googlenet_testing = models.googlenet(pretrained=True)
num_ftrs = googlenet_testing.fc.in_features
googlenet_testing.fc = nn.Linear(num_ftrs, 193)

googlenet_testing.load_state_dict(torch.load('./models/best_model_zindi_googlenet_26-11.pt'))

Downloading: "https://download.pytorch.org/models/googlenet-1378be20.pth" to /root/.cache/torch/hub/checkpoints/googlenet-1378be20.pth


<All keys matched successfully>

## Testing

First, I test each of the three models...

In [38]:
preds_densenet = test_model(densenet_testing, test_dl, device)
preds_resnet = test_model(resnet_testing, test_dl, device)
preds_googlenet = test_model(googlenet_testing, test_dl, device)

... then I put the test predictions of each model in three separate dataframes.

In [39]:
df_densenet = preds_to_dataframe(preds_densenet, test_paths, column_order, labels_numeric_mapping)
df_resnet = preds_to_dataframe(preds_resnet, test_paths, column_order, labels_numeric_mapping)
df_googlenet = preds_to_dataframe(preds_googlenet, test_paths, column_order, labels_numeric_mapping)

### Ensemble

Lastly, I average the predicted probability of each class per audio file and thus receiev my final submission dataframe, which I save as a CSV file, ready to be submitted.

In [44]:
dfs = [df_densenet, df_resnet, df_googlenet]
df_ensemble = pd.concat(dfs, ignore_index=False)
df_ensemble = df_ensemble.groupby('fn').mean().reset_index()
df_ensemble.head()

,fn,maize streak virus,disease,okukkoola,muwogo,mpeke,mucungwa,greens,garden,mango,bulimi,obuwuka,ebikoola,obulimi,ebisoolisooli,kaamulali,eddagala,beans,omuyembe,leaf,kisaanyi,leaves,butterfly,okuzifuuyira,micungwa,ppaapaali,emboga,kikolo,harvest,olusuku,coffee,super grow,rice,ensujju,okulima,worm,ebbugga,onion,ensigo,plantation,ensukusa,maize stalk borer,amakoola,muceere,insect,pumpkin,spray,namuginga,tomatoes,akatungulu,ground nuts,orange,nursery bed,npk,mangoes,ebitooke,maize,endwadde,lumonde,ensiringanyi,pawpaw,ebijjanjalo,sukumawiki,caterpillars,matooke seedlings,okusaasaana,omuceere,ebikajjo,mulimi,sikungula,watermelon,ennima,akammwanyi,amakungula,ebyobulimi,okugimusa,sow,Pump,ekigimusa,ekirime,okusiga,nfukirira,faamu,okulimibwa,pampu,vegetables,lusuku,ennyaanya,ekisaanyi,bibala,ekikajjo,ennimiro,okukungula,akatunda,eppapaali,endagala,ekijanjaalo,abalimi,plant,Spinach,ndwadde,caterpillar,miceere,farmer,irrigate,nnakati,omusiri,weeding,okusaasaanya,amalagala,ekikoola,ebisooli,munyeera,ekirwadde,enkota,banana,fruit picking,ebibala,dig,doodo,omulimisa,eggobe,seed,crop,okunnoga,obutunda,fruit,endokwa,ebikolo,omucungwa,okufukirira,obulwadde,ebijanjaalo,matooke,pepper,agriculture,afukirira,nnyaanya,medicine,nambaale,cassava,sugarcane,ekimera,kikajjo,obusaanyi,ekibala,ebigimusa,kukungula,nnimiro,ebikongoliro,irish potatoes,ensuku,emicungwa,ekikolo,ejjobyo,omulimi,okusimba,sweet potatoes,okufuuyira,farming instructor,nnasale beedi,passion fruit,ekitooke,ebisaanyi,ekyeya,enva endiirwa,emisiri,emiyembe,amatooke,ebiwuka,farm,ebinyebwa,amappapaali,ebimera,kassooli,harvesting,emmwanyi,akamonde,obumonde,cabbages,akasaanyi,spread,ebirime,drought,kasaanyi,suckers,insects,fertilizer,nakavundira,ekiwojjolo,akawuka,ddagala,ebiwojjolo,obutungulu
0,audio_files/00118N3.wav,1.655901e-02,2.566184e-04,0.000983,7.815802e-06,5.451176e-05,1.229861e-04,6.080459e-03,3.968441e-04,1.817760e-06,5.475927e-07,0.003923,1.102328e-05,6.836563e-07,3.574508e-04,0.000618,7.047406e-04,2.928811e-03,0.000014,5.925409e-03,3.470239e-02,5.508204e-07,6.870831e-04,1.530336e-03,0.000429,0.000219,0.000020,1.144012e-04,2.264158e-04,7.631443e-05,3.005815e-04,1.517564e-04,6.622267e-03,6.258010e-05,0.011023,2.063236e-04,0.001509,9.714249e-06,6.802850e-06,0.000209,6.787836e-04,1.853498e-02,9.005512e-05,0.000038,1.263762e-03,4.081209e-02,1.326979e-01,2.094195e-04,1.038423e-04,9.077267e-05,1.511208e-04,1.949672e-05,4.288993e-04,8.415258e-03,1.898771e-05,0.000012,7.774329e-06,5.915643e-04,0.000003,3.138577e-04,3.497244e-02,0.000011,7.749909e-04,8.239368e-03,2.518809e-03,1.684943e-03,0.000058,0.000098,3.420571e-06,1.990317e-05,3.037624e-03,0.000026,0.000036,1.655057e-05,8.663476e-07,2.118635e-02,5.370909e-05,2.905820e-04,3.498449e-03,0.000030,5.576209e-05,2.828234e-04,1.035643e-05,6.260152e-04,3.420281e-06,5.766856e-02,1.796864e-05,0.000013,2.879371e-03,0.000022,1.702958e-04,7.643644e-07,1.022903e-05,0.000489,0.000806,0.000009,4.481495e-03,0.000003,5.840157e-04,3.853690e-02,1.325241e-03,3.247826e-01,9.943552e-05,2.164849e-03,3.193091e-05,2.535203e-05,7.582310e-06,1.002599e-06,1.036530e-03,0.000054,3.965112e-05,0.000017,0.000025,1.302004e-03,0.000536,2.329407e-03,9.359569e-05,6.091113e-06,1.475349e-04,0.000020,2.978411e-05,9.176400e-05,2.051556e-06,6.861512e-03,1.016836e-04,0.000095,2.088893e-05,4.371952e-05,2.130980e-05,3.646415e-05,5.412077e-04,3.162383e-04,0.000014,6.582258e-06,1.960323e-02,1.008134e-03,0.000493,0.000023,5.134062e-03,0.000036,1.397984e-02,5.294014e-03,7.950900e-05,1.811183e-03,1.890884e-04,0.000054,1.489041e-03,2.765338e-06,2.246608e-05,1.209424e-03,1.123092e-03,3.473339e-06,7.534902e-05,3.215619e-05,0.000042,0.000002,0.001076,7.535356e-03,1.146381e-03,2.081413e-04,3.803876e-05,4.714733e-02,0.000242,3.092767e-03,1.517555e-05,0.000008,1.954656e-05,0.000009,0.000023,0.010375,3.142591e-05,0.000112,0.001382,1.941411e-06,0.000093,5.922175e-03,0.000007,0.000006,3.718511e-04,7.492611e-03,0.000980,1.170387e-02,7.816404e-07,1.700118e-03,0.000399,2.223352e-04,6.630169e-

In [46]:
df_ensemble.to_csv('./submissions/submission.csv', index=False)